Implementing diff Substitution Cipher Techniques like Caesar, Playfair, Hill and Vigenenre

In [ ]:
#S 1

# ======================== #
#     CAESAR CIPHER       #
# ======================== #

# A function to encrypt text using Caesar Cipher
def caesar_encrypt(text, s):
    result = ""  # Initialize the result string

    # Traverse through each character in the input text
    for i in range(len(text)):
        char = text[i]  # Get current character

        # If character is uppercase
        if (char.isupper()):
            # Shift character and append to result
            result += chr((ord(char) + s - 65) % 26 + 65)

        # If character is lowercase
        else:
            # Shift character and append to result
            result += chr((ord(char) + s - 97) % 26 + 97)

    return result  # Return the final encrypted text

# Sample text and shift
text = "ATTACKATONCE"
s = 4

# Display results
print("Text  : " + text)
print("Shift : " + str(s))
print("Cipher: " + caesar_encrypt(text, s))  # Encrypted text using Caesar Cipher


# ======================== #
#     PLAYFAIR CIPHER     #
# ======================== #

# Convert string to lowercase
def toLowerCase(text):
    return text.lower()

# Remove spaces from text
def removeSpaces(text):
    newText = ""
    for i in text:
        if i == " ":  # Skip spaces
            continue
        else:
            newText += i  # Add non-space character
    return newText

# Split text into pairs (digraphs)
def Diagraph(text):
    Diagraph = []
    group = 0
    for i in range(2, len(text), 2):  # Step by 2
        Diagraph.append(text[group:i])  # Add pair
        group = i
    Diagraph.append(text[group:])  # Add last pair
    return Diagraph

# Add filler letter 'x' if any digraph has repeated letters
def FillerLetter(text):
    k = len(text)
    if k % 2 == 0:  # Even length
        for i in range(0, k, 2):
            if text[i] == text[i+1]:  # Repeated letters
                new_word = text[0:i+1] + 'x' + text[i+1:]  # Insert filler
                new_word = FillerLetter(new_word)  # Recurse to check again
                break
            else:
                new_word = text
    else:  # Odd length
        for i in range(0, k-1, 2):
            if text[i] == text[i+1]:
                new_word = text[0:i+1] + 'x' + text[i+1:]
                new_word = FillerLetter(new_word)
                break
            else:
                new_word = text
    return new_word

# 5x5 Matrix excluding 'j'
list1 = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k',
         'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
         'u', 'v', 'w', 'x', 'y', 'z']

# Generate 5x5 key matrix from keyword
def generateKeyTable(word, list1):
    key_letters = []
    for i in word:
        if i not in key_letters:
            key_letters.append(i)  # Remove duplicates

    compElements = key_letters.copy()
    for i in list1:
        if i not in compElements:
            compElements.append(i)  # Fill rest of alphabet

    matrix = []
    while compElements != []:
        matrix.append(compElements[:5])  # Fill 5 characters per row
        compElements = compElements[5:]

    return matrix

# Locate letter in matrix
def search(mat, element):
    for i in range(5):
        for j in range(5):
            if mat[i][j] == element:
                return i, j

# Encrypt letters in same row
def encrypt_RowRule(matr, e1r, e1c, e2r, e2c):
    if e1c == 4:  # Wrap around
        char1 = matr[e1r][0]
    else:
        char1 = matr[e1r][e1c+1]

    if e2c == 4:
        char2 = matr[e2r][0]
    else:
        char2 = matr[e2r][e2c+1]

    return char1, char2

# Encrypt letters in same column
def encrypt_ColumnRule(matr, e1r, e1c, e2r, e2c):
    if e1r == 4:
        char1 = matr[0][e1c]
    else:
        char1 = matr[e1r+1][e1c]

    if e2r == 4:
        char2 = matr[0][e2c]
    else:
        char2 = matr[e2r+1][e2c]

    return char1, char2

# Encrypt letters in different row and column (rectangle rule)
def encrypt_RectangleRule(matr, e1r, e1c, e2r, e2c):
    char1 = matr[e1r][e2c]
    char2 = matr[e2r][e1c]
    return char1, char2

# Encrypt entire plaintext using Playfair cipher
def encryptByPlayfairCipher(Matrix, plainList):
    CipherText = []
    for i in range(len(plainList)):
        ele1_x, ele1_y = search(Matrix, plainList[i][0])
        ele2_x, ele2_y = search(Matrix, plainList[i][1])

        if ele1_x == ele2_x:
            c1, c2 = encrypt_RowRule(Matrix, ele1_x, ele1_y, ele2_x, ele2_y)
        elif ele1_y == ele2_y:
            c1, c2 = encrypt_ColumnRule(Matrix, ele1_x, ele1_y, ele2_x, ele2_y)
        else:
            c1, c2 = encrypt_RectangleRule(Matrix, ele1_x, ele1_y, ele2_x, ele2_y)

        CipherText.append(c1 + c2)
    return CipherText

# Main logic for Playfair
text_Plain = 'instruments'
text_Plain = removeSpaces(toLowerCase(text_Plain))
PlainTextList = Diagraph(FillerLetter(text_Plain))
if len(PlainTextList[-1]) != 2:
    PlainTextList[-1] += 'z'  # Padding if odd length

key = "Monarchy"
print("Key text:", key)
Matrix = generateKeyTable(toLowerCase(key), list1)
print("Plain Text:", text_Plain)

CipherList = encryptByPlayfairCipher(Matrix, PlainTextList)

# Join list to string
CipherText = "".join(CipherList)
print("CipherText:", CipherText)


# ======================== #
#       HILL CIPHER        #
# ======================== #

# Initialize 3x3 matrices
keyMatrix = [[0]*3 for _ in range(3)]
messageVector = [[0] for _ in range(3)]
cipherMatrix = [[0] for _ in range(3)]

# Convert key string to key matrix
def getKeyMatrix(key):
    k = 0
    for i in range(3):
        for j in range(3):
            keyMatrix[i][j] = ord(key[k]) % 65  # Convert A-Z to 0-25
            k += 1

# Matrix multiplication for encryption
def encrypt(messageVector):
    for i in range(3):
        for j in range(1):
            cipherMatrix[i][j] = 0
            for x in range(3):
                cipherMatrix[i][j] += (keyMatrix[i][x] * messageVector[x][j])
            cipherMatrix[i][j] %= 26  # Modulo 26 to stay in A-Z

# Encrypt using Hill Cipher
def HillCipher(message, key):
    getKeyMatrix(key)  # Generate key matrix

    for i in range(3):
        messageVector[i][0] = ord(message[i]) % 65  # Convert message to vector

    encrypt(messageVector)  # Encrypt the vector

    CipherText = []
    for i in range(3):
        CipherText.append(chr(cipherMatrix[i][0] + 65))  # Convert to letters

    print("Ciphertext: ", "".join(CipherText))

# Encrypt using Hill Cipher
message = "ACT"
key = "GYBNQKURP"
HillCipher(message, key)


# ============================ #
#     VIGENÈRE CIPHER          #
# ============================ #

# Generate key of equal length to the message
def generate_key(msg, key):
    key = list(key)
    if len(msg) == len(key):
        return key
    else:
        for i in range(len(msg) - len(key)):
            key.append(key[i % len(key)])
    return "".join(key)

# Encrypt message using Vigenère Cipher
def encrypt_vigenere(msg, key):
    encrypted_text = []
    key = generate_key(msg, key)
    for i in range(len(msg)):
        char = msg[i]
        if char.isupper():
            encrypted_char = chr((ord(char) + ord(key[i]) - 2 * ord('A')) % 26 + ord('A'))
        elif char.islower():
            encrypted_char = chr((ord(char) + ord(key[i]) - 2 * ord('a')) % 26 + ord('a'))
        else:
            encrypted_char = char
        encrypted_text.append(encrypted_char)
    return "".join(encrypted_text)

# Decrypt Vigenère cipher text
def decrypt_vigenere(msg, key):
    decrypted_text = []
    key = generate_key(msg, key)
    for i in range(len(msg)):
        char = msg[i]
        if char.isupper():
            decrypted_char = chr((ord(char) - ord(key[i]) + 26) % 26 + ord('A'))
        elif char.islower():
            decrypted_char = chr((ord(char) - ord(key[i]) + 26) % 26 + ord('a'))
        else:
            decrypted_char = char
        decrypted_text.append(decrypted_char)
    return "".join(decrypted_text)

# Example usage of Vigenère Cipher
text_to_encrypt = "Hello, World!"
key = "KEY"
encrypted_text = encrypt_vigenere(text_to_encrypt, key)
print(f"Encrypted Text: {encrypted_text}")
decrypted_text = decrypt_vigenere(encrypted_text, key)
print(f"Decrypted Text: {decrypted_text}")


In [1]:
#G 1

# CaesarCipher class implements the Caesar Cipher encryption and decryption techniques.
class CaesarCipher:

    # Method to encrypt the text using the Caesar Cipher technique with a given shift value.
    def encrypt(self, txt, shift):
        res = ""  # Initialize an empty string to store the encrypted text

        # Loop through each character in the input text
        for char in txt:
            if char.isupper():  # Check if the character is an uppercase letter
                # Encrypt the character by shifting within the uppercase range ('A' to 'Z')
                res += chr((ord(char) + shift - 65) % 26 + 65)
            elif char.islower():  # Check if the character is a lowercase letter
                # Encrypt the character by shifting within the lowercase range ('a' to 'z')
                res += chr((ord(char) + shift - 97) % 26 + 97)
            else:
                # If the character is neither uppercase nor lowercase (e.g., punctuation or spaces),
                # leave it unchanged in the result string
                res += char

        return res  # Return the encrypted string after the loop

    # Method to decrypt the text by shifting the characters in the opposite direction (using negative shift).
    def decrypt(self, txt, shift):
        return self.encrypt(txt, -shift)  # Decryption is simply the reverse of encryption (negative shift)

class PlayfairCipher:

    # Convert the given text to lowercase.
    def toLowerCase(self, txt):
        return txt.lower()

    # Remove all spaces from the input text.
    def removeSpaces(self, txt):
        return "".join([i for i in txt if i != " "])

    # Split the input text into pairs of characters (digraphs).
    def Diagraph(self, txt):
        if not txt:  # If the text is empty, return an empty list
            return []

        pairs = []
        i = 0
        while i < len(txt):
            # If there are two characters left, add the pair.
            if i + 1 < len(txt):
                pairs.append(txt[i:i + 2])
                i += 2
            else:
                # If there is only one character left, append an 'x' to make it a pair.
                pairs.append(txt[i] + "x")  # Add filler letter for odd length
                i += 1

        return pairs

    # Add a filler letter ('x') to the message if its length is odd.
    def FillerLetter(self, txt):
        if len(txt) % 2 == 0:
            return txt  # Return as is if the length is already even
        return txt + 'x'  # Add 'x' if the length is odd

    # Generate the 5x5 matrix key table based on the provided key.
    def genKeyTable(self, key, alphabet):
        mat = []
        key = ''.join(dict.fromkeys(key))  # Remove duplicates from the key
        key += ''.join([i for i in alphabet if i not in key])  # Add remaining alphabet letters not in the key
        for i in range(5):
            mat.append(list(key[i*5:(i+1)*5]))  # Create a 5x5 matrix

        return mat

    # Search for the position of a character in the key table.
    def search(self, mat, char):
        for i in range(5):
            for j in range(5):
                if mat[i][j] == char:
                    return i, j  # Return the row and column indices of the character

    # Encrypt the message using the Playfair Cipher technique.
    def encrypt(self, msg, key):
        if not msg:
            return "Error: Empty message cannot be encrypted"

        alphabet = "abcdefghijklmnopqrstuvwxyz"
        msg = self.removeSpaces(self.toLowerCase(msg))  # Convert to lowercase and remove spaces
        msg = self.FillerLetter(msg)  # Add filler letter if needed

        mat = self.genKeyTable(key, alphabet)  # Generate the key table
        pairs = self.Diagraph(msg)  # Split the message into digraphs

        if not pairs:
            return "Error: Message couldn't be split into valid digraphs"

        ciphertxt = []

        # Process each pair of characters
        for pair in pairs:
            x1, y1 = self.search(mat, pair[0])  # Find the position of the first character
            x2, y2 = self.search(mat, pair[1])  # Find the position of the second character

            # Case 1: Same row, shift right
            if x1 == x2:
                ciphertxt.append(mat[x1][(y1 + 1) % 5])
                ciphertxt.append(mat[x2][(y2 + 1) % 5])
            # Case 2: Same column, shift down
            elif y1 == y2:
                ciphertxt.append(mat[(x1 + 1) % 5][y1])
                ciphertxt.append(mat[(x2 + 1) % 5][y2])
            # Case 3: Different row and column, form a rectangle (swap corners)
            else:
                ciphertxt.append(mat[x1][y2])
                ciphertxt.append(mat[x2][y1])

        return ''.join(ciphertxt)  # Return the encrypted text

    # Decrypt the message using the Playfair Cipher technique.
    def decrypt(self, msg, key):
        if not msg:
            return "Error: Empty message cannot be decrypted"

        alphabet = "abcdefghijklmnopqrstuvwxyz"
        msg = self.removeSpaces(self.toLowerCase(msg))  # Convert to lowercase and remove spaces
        mat = self.genKeyTable(key, alphabet)  # Generate the key table
        pairs = self.Diagraph(msg)  # Split the message into digraphs

        if not pairs:
            return "Error: Message couldn't be split into valid digraphs"

        plaintxt = []

        # Process each pair of characters
        for pair in pairs:
            x1, y1 = self.search(mat, pair[0])  # Find the position of the first character
            x2, y2 = self.search(mat, pair[1])  # Find the position of the second character

            # Case 1: Same row, shift left
            if x1 == x2:
                plaintxt.append(mat[x1][(y1 - 1) % 5])
                plaintxt.append(mat[x2][(y2 - 1) % 5])
            # Case 2: Same column, shift up
            elif y1 == y2:
                plaintxt.append(mat[(x1 - 1) % 5][y1])
                plaintxt.append(mat[(x2 - 1) % 5][y2])
            # Case 3: Different row and column, form a rectangle (swap corners)
            else:
                plaintxt.append(mat[x1][y2])
                plaintxt.append(mat[x2][y1])

        return ''.join(plaintxt)  # Return the decrypted text

class HillCipher:

    def getKeyMat(self, key):
        keyMat = [[0] * 3 for i in range(3)]
        k = 0
        for i in range(3):
            for j in range(3):
                keyMat[i][j] = ord(key[k]) % 65
                k += 1
        return keyMat

    def getMatInverse(self, mat):
        # Find determinant
        det = (mat[0][0] * (mat[1][1] * mat[2][2] - mat[1][2] * mat[2][1]) -
               mat[0][1] * (mat[1][0] * mat[2][2] - mat[1][2] * mat[2][0]) +
               mat[0][2] * (mat[1][0] * mat[2][1] - mat[1][1] * mat[2][0])) % 26

        # Find modular inverse of determinant
        det_inv = pow(det, -1, 26)

        # Find inverse matrix using the adjugate matrix method
        invMat = [
            [
                (mat[1][1] * mat[2][2] - mat[1][2] * mat[2][1]) * det_inv % 26,
                (mat[0][2] * mat[2][0] - mat[0][0] * mat[2][2]) * det_inv % 26,
                (mat[0][1] * mat[1][0] - mat[0][0] * mat[1][1]) * det_inv % 26,
            ],
            [
                (mat[1][2] * mat[2][0] - mat[1][0] * mat[2][2]) * det_inv % 26,
                (mat[0][0] * mat[2][2] - mat[0][2] * mat[2][0]) * det_inv % 26,
                (mat[0][1] * mat[2][0] - mat[0][0] * mat[2][1]) * det_inv % 26,
            ],
            [
                (mat[1][0] * mat[2][1] - mat[1][1] * mat[2][0]) * det_inv % 26,
                (mat[0][1] * mat[2][0] - mat[0][0] * mat[2][1]) * det_inv % 26,
                (mat[0][0] * mat[1][1] - mat[0][1] * mat[1][0]) * det_inv % 26,
            ]
        ]
        return invMat

    def encrypt(self, msg, key):
        # Pad the message if its length is not a multiple of 3
        while len(msg) % 3 != 0:
            msg += 'X'  # Using 'X' as the filler character

        keyMat = self.getKeyMat(key)
        cipherText = []

        # Process each block of 3 letters
        for i in range(0, len(msg), 3):
            msgVec = [ord(msg[i]) % 65, ord(msg[i + 1]) % 65, ord(msg[i + 2]) % 65]
            cipherMat = [0, 0, 0]

            for j in range(3):
                cipherMat[j] = sum(keyMat[j][x] * msgVec[x] for x in range(3)) % 26

            cipherText.append(''.join([chr(c + 65) for c in cipherMat]))

        return ''.join(cipherText)

    def decrypt(self, msg, key):
        keyMat = self.getKeyMat(key)
        invMat = self.getMatInverse(keyMat)
        plainText = []

        # Process each block of 3 letters
        for i in range(0, len(msg), 3):
            msgVec = [ord(msg[i]) % 65, ord(msg[i + 1]) % 65, ord(msg[i + 2]) % 65]
            plainMat = [0, 0, 0]

            for j in range(3):
                plainMat[j] = sum(invMat[j][x] * msgVec[x] for x in range(3)) % 26

            plainText.append(''.join([chr(c + 65) for c in plainMat]))

        return ''.join(plainText)

# Example usage
cipher = HillCipher()
key = "GYBNQKURP"  # Example key
message = "HELLO"

encrypted_msg = cipher.encrypt(message, key)
print("Encrypted Message:", encrypted_msg)

decrypted_msg = cipher.decrypt(encrypted_msg, key)
print("Decrypted Message:", decrypted_msg)

class VigenereCipher:

    def gen_key(self, msg, key):
        # Ensure the key is non-empty
        if not key:
            raise ValueError("Key cannot be empty.")

        key = list(key)
        if len(msg) == len(key):
            return key

        for i in range(len(msg) - len(key)):
            key.append(key[i % len(key)])  # Repeat key if necessary

        return "".join(key)

    def encrypt(self, msg, key):
        encrypted_txt = []
        key = self.gen_key(msg, key)  # Generate the key based on the message length

        for i in range(len(msg)):
            char = msg[i]

            if char.isupper():
                encrypted_char = chr((ord(char) + ord(key[i].upper()) - 2 * ord('A')) % 26 + ord('A'))
            elif char.islower():
                encrypted_char = chr((ord(char) + ord(key[i].lower()) - 2 * ord('a')) % 26 + ord('a'))
            else:
                encrypted_char = char  # Non-alphabetic characters remain unchanged
            encrypted_txt.append(encrypted_char)

        return "".join(encrypted_txt)

    def decrypt(self, msg, key):
        decrypted_txt = []
        key = self.gen_key(msg, key)  # Generate the key based on the message length

        for i in range(len(msg)):
            char = msg[i]

            if char.isupper():
                decrypted_char = chr((ord(char) - ord(key[i].upper()) + 26) % 26 + ord('A'))
            elif char.islower():
                decrypted_char = chr((ord(char) - ord(key[i].lower()) + 26) % 26 + ord('a'))
            else:
                decrypted_char = char  # Non-alphabetic characters remain unchanged
            decrypted_txt.append(decrypted_char)

        return "".join(decrypted_txt)

# Example usage
cipher = VigenereCipher()
key = "KEY"
message = "Hello World!"

encrypted_msg = cipher.encrypt(message, key)
print("Encrypted Message:", encrypted_msg)

decrypted_msg = cipher.decrypt(encrypted_msg, key)
print("Decrypted Message:", decrypted_msg)

def main():

    while True:
        print("\nSelect Cipher Technique:")
        print("1. Caesar Cipher")
        print("2. Playfair Cipher")
        print("3. Hill Cipher")
        print("4. Vigenere Cipher")
        print("5. Exit")
        ch = input("Enter choice: ")

        if ch == "1":
            cipher = CaesarCipher()
            txt = input("Enter text to encrypt/decrypt: ")
            shift = int(input("Enter the shift value: "))
            action = input("Do you want to (E)ncrypt or (D)ecrypt? ").lower()

            if action == "e":
                print("Encrypted Text:", cipher.encrypt(txt, shift))
            elif action == "d":
                print("Decrypted Text:", cipher.decrypt(txt, shift))

        elif ch == "2":
            cipher = PlayfairCipher()
            txt = input("Enter text to encrypt: ")
            key = input("Enter key: ")
            print("Encrypted Text:", cipher.encrypt(txt, key))

        elif ch == "3":
            cipher = HillCipher()
            txt = input("Enter text to encrypt (3 letters): ")
            key = input("Enter 9-character key: ")
            print("Encrypted Text:", cipher.encrypt(txt, key))

        elif ch == "4":
            cipher = VigenereCipher()
            txt = input("Enter the text to encrypt/decrypt: ")
            key = input("Enter the key: ")
            action = input("Do you want to (E)ncrypt or (D)ecrypt? ").lower()

            if action == "e":
                print("Encrypted Text:", cipher.encrypt(txt, key))
            elif action == "d":
                print("Decrypted Text:", cipher.decrypt(txt, key))

        elif ch == "5":

            break

        else:
            print("Invalid choice, please try again.")

if __name__ == "__main__":
    main()


Encrypted Message: TFJJZX
Decrypted Message: OBLAPX
Encrypted Message: Rijvs Gspvh!
Decrypted Message: Hello World!

Select Cipher Technique:
1. Caesar Cipher
2. Playfair Cipher
3. Hill Cipher
4. Vigenere Cipher
5. Exit
Enter choice: 1
Enter text to encrypt/decrypt: Hello Khelesh
Enter the shift value: 6
Do you want to (E)ncrypt or (D)ecrypt? E
Encrypted Text: Nkrru Qnkrkyn

Select Cipher Technique:
1. Caesar Cipher
2. Playfair Cipher
3. Hill Cipher
4. Vigenere Cipher
5. Exit
Enter choice: 2
Enter text to encrypt: Hello Khelesh
Enter key: Devil
Encrypted Text: obDDqhobDvtg

Select Cipher Technique:
1. Caesar Cipher
2. Playfair Cipher
3. Hill Cipher
4. Vigenere Cipher
5. Exit
Enter choice: 3
Enter text to encrypt (3 letters): God
Enter 9-character key: Godfather
Encrypted Text: XVR

Select Cipher Technique:
1. Caesar Cipher
2. Playfair Cipher
3. Hill Cipher
4. Vigenere Cipher
5. Exit
Enter choice: 4
Enter the text to encrypt/decrypt: Hello Khelesh
Enter the key: Bless
Do you want to (E)

In [ ]:
"""
### **Concept Notes and Technicalities of Cipher Techniques:**

In the context of cryptography, ciphers are methods used to encode or encrypt messages to keep them secure from unauthorized access. The general goal of a cipher is to obscure the original content of the message, making it unintelligible to anyone who doesn't have the decryption key.

Here’s a breakdown of key concepts, especially focusing on key shifts in ciphers:

---

### **1. Caesar Cipher:**
The Caesar Cipher is one of the simplest and most well-known encryption techniques.

#### **Key Shift:**
- A **key shift** (or **shift** value) in the Caesar Cipher refers to the number of positions each letter in the plaintext is shifted to generate the ciphertext.

  **For example**:
  - With a key of `3`, each letter of the plaintext is shifted 3 positions forward in the alphabet:
    - Plaintext: `A B C D E F`
    - Ciphertext: `D E F G H I`

- The shift wraps around the alphabet (for instance, if you shift 'Z' by 1, it becomes 'A').

#### **Key shift in technical terms**:
- **Encryption formula**: `E(x) = (x + shift) % 26`
  - `x` is the position of the letter in the alphabet.
  - `shift` is the number of positions to move.
  - `26` is used because there are 26 letters in the alphabet.

### **2. Vigenère Cipher:**
The Vigenère cipher is a more advanced form of substitution cipher that uses a keyword to determine the shift for each letter.

#### **Key Shift in Vigenère Cipher**:
- The shift for each letter is determined by the corresponding letter in the key.
- If the plaintext is shorter than the key, the key repeats.

  **For example**:
  - Plaintext: `HELLO`
  - Key: `KEYKEY`
  - Ciphertext: Varying shifts for each letter based on the corresponding key letter.

#### **Key shift in technical terms**:
- **Encryption formula**: `E(x) = (P + K) % 26`
  - `P` is the position of the plaintext letter.
  - `K` is the position of the key letter.
  - `26` is used for the alphabet wraparound.

  So, for the letter 'H' and key 'K' (`K` = 10), the shift would be `E(H) = (7 + 10) % 26 = 17`, resulting in the letter 'R'.

### **3. Playfair Cipher:**
The Playfair cipher encrypts digraphs (pairs of letters) instead of individual characters.

#### **Key Shift in Playfair Cipher**:
- A key table is constructed from the keyword and the alphabet, where each pair of letters is mapped onto the table in a grid.
- The shift for each letter in a digraph depends on their positions in the grid.

  **Example**:
  - Plaintext: `HELLO`
  - Key: `KEYWORD`
  - The ciphertext is formed by shifting the letters in the digraphs based on their position in the key table.

#### **Key shift in technical terms**:
- If both letters of a digraph appear in the same row or column of the grid, they are replaced by the letter immediately to their right (wrap around if necessary).
- If they appear in different rows and columns, each letter is replaced by the letter in the same row but the column of the other letter in the pair.

### **4. Hill Cipher:**
The Hill cipher is a polygraphic substitution cipher that encrypts text using a matrix (of size n x n, typically 2x2 or 3x3).

#### **Key Shift in Hill Cipher**:
- The key is a square matrix that defines the transformation applied to the plaintext letters.
- The text is grouped into vectors (e.g., pairs or triplets of letters), and each vector is multiplied by the key matrix.

#### **Key shift in technical terms**:
- The encryption process is a matrix multiplication.
  - **Encryption formula**: `C = K * P` (mod 26)
  - `C` is the ciphertext, `K` is the key matrix, and `P` is the plaintext vector.

- The shift here refers to the transformation of each character based on its position in the matrix.

---

### **General Concepts Related to Cipher Key Shift:**

#### **1. Modular Arithmetic:**
- Many ciphers use **modular arithmetic** (modulo 26 in the case of letters) to handle the wraparound of the alphabet.
- This ensures that when you shift past 'Z', the cipher "wraps" back to 'A'.

  **Example**:
  - Shifting 'Z' by 1 gives: `(ord('Z') + 1) % 26` which wraps around to 'A'.

#### **2. Decryption:**
- Decryption is simply the inverse of encryption. For ciphers like the Caesar cipher, it involves shifting the letters backward by the same key value.

  **For example**, for Caesar Cipher with a shift of `3`, decryption would shift letters back by `3` positions.

#### **3. Frequency Analysis:**
- A common method for breaking simple ciphers like Caesar and Vigenère is **frequency analysis**. In this method, the frequency of each letter in the ciphertext is analyzed, assuming that certain letters (like 'e' in English) are more common than others.

  - If the cipher uses a simple shift (like Caesar), the frequency of letters in the ciphertext will look similar to the frequency in the original plaintext.

#### **4. Key Strength:**
- The strength of a cipher is often related to the length of the key (in terms of randomness and complexity).
  - **Short keys** (like in the Caesar cipher) are easy to break.
  - **Longer keys** (like in Vigenère or Hill cipher) make the cipher stronger as it becomes more difficult to guess or brute-force.

---

### **Conclusion:**
- **Key Shift** in ciphers generally refers to how much each letter is moved (shifted) in the alphabet or how the transformation occurs based on a matrix or a key. It's the core mechanism behind many encryption algorithms, whether simple (like Caesar) or more complex (like Vigenère and Hill).
- Understanding key shift helps in analyzing cipher techniques and provides insights into how encryption and decryption processes work.
"""
